### SDF Download with Custom Filenames & Download Links

In [ ]:
import pandas as pd
import requests
import os
import time
import re

# Load your Excel file
df = pd.read_excel(r"D:\TMH\Proton_Lab\Gokul_Project\Boswellia Serrata Phytochemicals.xlsx")  # Replace with your actual filename

# Create output folder
output_dir = r"D:\TMH\Proton_Lab\Gokul_Project\sdf_files"
os.makedirs(output_dir, exist_ok=True)

# Store download links
download_links = []

# Loop through each row
for index, row in df.iterrows():
    compound = str(row["Phytochemical Name"]).strip()
    imp_id = str(row["IMPPAT ID"]).strip()
    pubchem_id = str(row["PubChem ID"]).strip()

    # Extract numeric CID and clean compound name
    match = re.search(r"CID:(\d+)", pubchem_id)
    if not match:
        download_links.append("NotFound")
        continue

    cid = match.group(1)
    clean_name = re.sub(r"^[^a-zA-Z0-9]+", "", compound)

    # Create filename
    filename = f"{clean_name}_{imp_id}_{cid}.sdf"
    file_path = os.path.join(output_dir, filename)

    # PubChem download URL
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/SDF?record_type=3d"

    # Download SDF
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(file_path, "wb") as f:
                f.write(response.content)
            print(f"Saved: {filename}")
            # Add clickable link
            download_links.append(f'=HYPERLINK("{url}", "Download")')
        else:
            print(f"Failed CID {cid} (status {response.status_code})")
            download_links.append("Failed")
    except requests.exceptions.RequestException as e:
        print(f"Error for CID {cid}: {e}")
        download_links.append("Error")

    time.sleep(0.2)

# ➕ Add links to DataFrame
df["3D Structure Link"] = download_links

# Save to new CSV
df.to_csv(r"D:\TMH\Proton_Lab\Gokul_Project\imppat_with_download_links.csv", index=False)
print("CSV saved as 'imppat_with_download_links.csv'")
